In [ ]:
!pip install git+https://github.com/nasa/pretrained-microscopy-models

In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import torch
import gc
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import math

from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.model_selection import train_test_split, LeaveOneOut, cross_val_score, KFold
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import make_scorer,r2_score
from sklearn.svm import SVR

from transformers import CLIPProcessor, CLIPModel, AutoModel, CLIPImageProcessor, SamModel, SamProcessor, AutoProcessor

#import torch
import pretrained_microscopy_models as pmm
import torch.utils.model_zoo as model_zoo
import torch.nn as nn



In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)
url = pmm.util.get_pretrained_microscopynet_url('resnet50', 'micronet')


state_dict = torch.hub.load_state_dict_from_url(url, map_location=torch.device('cpu'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [5]:
transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                 ])

In [6]:
def make_NASA_features(img):
    # print (img.shape)
    # plt.imshow(img)
    # plt.show()

    x_shape = img.shape[0]
    y_shape = img.shape[1]

    if x_shape <= y_shape:
      min = x_shape
    else:
      min = y_shape

    img_new = img[:min, :min]

    img_rgb = Image.fromarray(img_new*255).convert('RGB')

    #print (img_rgb.size)

    img_t = transform(img_rgb)

    # image = img_t.permute(1, 2, 0)
    # plt.imshow(image)
    # plt.show()

    img_t = img_t.unsqueeze(0)

    # Extract everything up to the avgpool layer
    feature_extractor = nn.Sequential(*list(model.children())[:-1])
    feature_extractor.eval()


    with torch.no_grad():
         features = feature_extractor(img_t)
    features = np.array(features)
    features = np.squeeze(features)

    #print (features.shape)
    return features

In [ ]:
xx_bin = np.load('/content/drive/MyDrive/11_12/xx.npy')
xy_bin = np.load('/content/drive/MyDrive/11_12/xy.npy')
xz_bin = np.load('/content/drive/MyDrive/11_12/xz.npy')
y = np.load('/content/drive/MyDrive/11_12/digital_y.npy')

print (y[:10])

In [ ]:
import time
start_time = time.time()
count  = 0
xx_features = []
for x in xx_bin:

    xx = make_NASA_features(x)
    xx_features.append(xx)
    #print (count)
    count = count + 1

xx_features = np.array(xx_features)
new_filename =  '/content/drive/MyDrive/11_12/11_12_Digital_Features/xx_Nasa_2.npy'
np.save(new_filename, xx_features)

print ("DONE WITH XX")

count  = 0
xy_features = []
for y in xy_bin:

    xy = make_NASA_features(y)
    xy_features.append(xy)
    #print (count)
    count = count + 1

xy_features = np.array(xy_features)

new_filename =  '/content/drive/MyDrive/11_12/11_12_Digital_Features/xy_Nasa_2.npy'
np.save(new_filename, xy_features)

print ("DONE WITH XY")

count  = 0
xz_features = []
for z in xz_bin:

    xz = make_NASA_features(z)
    xz_features.append(xz)
    #print (count)
    count = count + 1

xz_features = np.array(xz_features)

new_filename =  '/content/drive/MyDrive/11_12/11_12_Digital_Features/xz_Nasa_2.npy'
np.save(new_filename, xz_features)

print ("DONE WITH XZ")
print("--- %s seconds ---" % (time.time() - start_time))